In [ ]:
import tensorflow.compat.v1 as tf
import tensorflow as tf2
tf.disable_v2_behavior()

# tf.Graph()

`tf.Graph()`

**Docstring**

创建一个表示 TensorFlow 计算时数据流的空的有向图，即计算图。

计算图通过众多`tf.function`来表示函数运算，每个计算图由一系列`tf.Operation`和`tf.Tensor`的节点组成，前者表示计算单元，后者表示计算之间流动的数据；计算图中的边也有两种类型，一种代表数据流动的方向，另一种则是单纯代表节点执行顺序；默认计算图则可以使用`tf.Graph.as_default`的上下文管理器来注册，随后计算等操作将被添加到计算图中，但并不立即执行；默认计算图也可以通过`tf.compat.v1.get_default_graph()`获得；一个`tf.Graph`实例支持任意数量的“collection”（有关 collection 详见`colleciton`），这些 collection 通过名称进行标识；在构建大型图时，为了方便起见，一个集合可以存储许多相关对象，如`tf.Variable`类会使用`tf.GraphKeys.GLOBAL_VARIABLES`集合来表示在计算图构造期间创建的所有变量；使用者可以通过指定新名称来定义其他集合；

通过计算图来执行计算，可以方便的保留中间节点的计算结果，进而有助于反向传播时链式法则求导；

TensorFlow 1中`tf.Graph`可以在不使用`tf.function`情况下直接构建和使用，但在 TensorFlow 2 中这种使用方式已经被摒弃，进而更建议通过`tf.function`来构建。如果直接使用`tf.Graph`，则还应使用其他 TensorFlow 1 的类来执行该计算图，如`tf.compat.v1.Session`等；

**note**: `tf.Graph`类对于计算图构造来说不是线程安全的；所有操作都应该从单个线程创建，否则必须提供外部同步；除非另有说明，否则所有方法都不是线程安全的；

**Type**

type

In [ ]:
g = tf.Graph()
with g.as_default():
    x = tf.constant(30.0)
    print(x.graph == g)

tensorboard 可视化

In [ ]:
tf.reset_default_graph()
x = tf.constant(3, name="x")
y = tf.constant(2, name="y")
z = tf.add(x, y, name="add_x_y")
writer = tf.summary.FileWriter("../009_TensorBoard/graphs/test1", tf.get_default_graph())
writer.close()

## tf.Graph().get_collection()

`g.get_colleciton((self, name, scope=None))`

**Docstring**

　　TensorFlow 的 collection 提供一个全局的存储机制，不会受到变量命名空间的影响，便于获得指定类型的变量

　　返回某个集合中具有给定`name`的列表，当没有值添加到该集合时则返回空列表；列表中元素以其被附加进去的顺序排列；这与`get_collection_ref()`不同，因为每次调用`get_collection_ref()`都会返回一个新列表，故其总是返回实际的集合列表(如果存在的话)

　　**`tf.get_collection(key, scope=None)`**是此函数的包装，其中参数`key`对应这里的参数`name`，使用方式相同

**Args**

- name: 集合的键，`GraphKeys`类包含许多集合的标准名称

- scope: 当指明时，那些`name`与正在调用的`re.match`相匹配的项才会返回到结果列表之中，那些没有`name`的项将永远不会被返回；`re.match`意味着没有特殊标记的`scope`将通过前缀来筛选

**File**: \tensorflow\lib\site-packages\tensorflow\python\framework\ops.py

**Type**: function

In [26]:
tf.reset_default_graph()
def dense_relu(x, size, phase_train, scope, reuse=True):
    with tf.variable_scope(scope):
        weights = tf.Variable(tf.random_normal([x.shape[-1], size]), name="weights")
        bias = tf.Variable(tf.random_normal([size]), name="bias")
        h = tf.add(tf.matmul(x, weights, name="matrix_mutiply"), bias, name="add_w_b")
        return tf.nn.relu(h, name='relu')

reuse = True
phase = False
x_inp_1 = tf.placeholder("float", shape=[None, 4096])
x_inp_2 = tf.placeholder("float", shape=[None, 4096])
x_inp_3 = tf.placeholder("float", shape=[None, 4096])
x_inp_4 = tf.placeholder("float", shape=[None, 4096])
x = (x_inp_1, x_inp_2, x_inp_3, x_inp_4)
with tf.variable_scope("network"):
    # first layer: 1024 -> 512
    h1_1 = dense_relu(x[0], 512, phase, 'layer1', reuse=reuse)
    h1_2 = dense_relu(x[1], 512, phase, 'layer1')
    h1_3 = dense_relu(x[2], 512, phase, 'layer1')
    h1_4 = dense_relu(x[3], 512, phase, 'layer1')
    # second layer: 512 -> 256
    h2_1 = dense_relu(h1_1, 256, phase, 'layer2', reuse=reuse)
    h2_2 = dense_relu(h1_2, 256, phase, 'layer2')
    h2_3 = dense_relu(h1_3, 256, phase, 'layer2')
    h2_4 = dense_relu(h1_4, 256, phase, 'layer2')
    # third layer: 256 -> 128 
    h3_1 = dense_relu(h2_1, 128, phase, 'layer3', reuse=reuse)
    h3_2 = dense_relu(h2_2, 128, phase, 'layer3')
    h3_3 = dense_relu(h2_3, 128, phase, 'layer3')
    h3_4 = dense_relu(h2_4, 128, phase, 'layer3')
    # aggregation layer: averaging
    h3 = h3_1 / 4. + h3_2 / 4. + h3_3 / 4. + h3_4 / 4.
    # fourth layer: 64 nodes
    h4 = dense_relu(h3, 64, phase, 'layer4', reuse=reuse)
writer = tf.summary.FileWriter("../009_Tensorboard/graphs/test1", tf.get_default_graph())
writer.close()

In [29]:
y = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)

## tf.Graph().control_dependencies()
`g.control_dependencies(control_inputs)`

**Docstring**

　　返回一个上下文管理器，为上下文中构造的所有操作指定控制依赖项 (control dependencies)，即`control_inputs`会在运行上下文中定义的**操作**之前先被执行，其中`control_inputs`为`Operation`或`Tensor`组成的列表，**常在需要指定代码先后运行顺序的情况下使用**；None 时则清空控制依赖项。`control_dependencies()`是可嵌套的。如果启用了即时执行，将调用`control_inputs`列表中的任何可调用对象
  
　　**`tf.control_dependencies(control_inputs)`**
是该函数的一个包装

**File**: \tensorflow\lib\site-packages\tensorflow\python\framework\ops.py

**Type**:      function

You can pass None to clear the control dependencies:

```python
with g.control_dependencies([a, b]):
  # Ops constructed here run after `a` and `b`.
  with g.control_dependencies(None):
    # Ops constructed here run normally, not waiting for either `a` or `b`.
    with g.control_dependencies([c, d]):
      # Ops constructed here run after `c` and `d`, also not waiting
      # for either `a` or `b`.
```

*N.B.* The control dependencies context applies *only* to ops that
are constructed within the context. Merely using an op or tensor
in the context does not add a control dependency. The following
example illustrates this point:

```python
# WRONG
def my_func(pred, tensor):
    t = tf.matmul(tensor, tensor)
    with tf.control_dependencies([pred]):
        # The matmul op is created outside the context, so no control dependency will be added.
        return t

# RIGHT
def my_func(pred, tensor):
  with tf.control_dependencies([pred]):
    # The matmul op is created in the context, so a control dependency
    # will be added.
    return tf.matmul(tensor, tensor)
```

注意，`tf.control_dependencies`确保在碰到上下文中的操作语句时先执行`control_inputs`，进而如果上下文中没有操作时，`control_inputs`将永远不会被执行

In [20]:
tf.reset_default_graph()
x = tf.Variable(0.0)
x_plus1 = tf.assign_add(x, 1)
 
with tf.control_dependencies([x_plus1]):
    y = x  # not an op

with tf.Session() as session:
    tf.global_variables_initializer().run()
    for i in range(5):
        print(y.eval(), end=", ")  # => 0.0, 0.0, 0.0, 0.0, 0.0,
    print()

with tf.control_dependencies([x_plus1]):
    y = tf.identity(x)
    
with tf.Session() as session:
    tf.global_variables_initializer().run()
    for i in range(5):
        print(y.eval(), end=", ")  # => 1.0, 2.0, 3.0, 4.0, 5.0,
    print()

0.0, 0.0, 0.0, 0.0, 0.0, 
1.0, 2.0, 3.0, 4.0, 5.0, 


虽然在`control_dependencies`作用域下创建的操作的执行将触发依赖项的执行，但该操作仍可能从正常的 tensorflow 计算图中被剪掉；如在下面的代码片段中，依赖项永远不会执行
```python
loss = model.loss()
with tf.control_dependencies(dependencies):
    loss = loss + tf.constant(1)  # dependencies ignored in the backward pass
return tf.gradients(loss, model.variables)
```
这是因为在运行梯度的计算图时不需要计算前向传递中的`constant(1)`操作

#  

#  

#  

#  

# tf.GraphKeys()

`tf.GraphKeys()`

**Docstring**

用于计算图集合的标准名称

The standard library uses various well-known names to collect and retrieve values associated with a graph. For example, the `tf.Optimizer` subclasses default to optimizing the variables collected under `tf.GraphKeys.TRAINABLE_VARIABLES` if none is specified, but it is also possible to pass an explicit list of variables.

The following standard keys are defined:

* `GLOBAL_VARIABLES`: `Variable`对象默认的集合，在分布式环境中共享，`MODEL_VARIABLES`属于这个的子集，更多细节详见`tf.compat.v1.global_variables`，通常情况下，所有的`TRAINABLE_VARIABLES`变量都属于`MODEL_VARIABLES`，所有`MODEL_VARIABLES`变量都属于`GLOBAL_VARIABLES`

* `LOCAL_VARIABLES`: the subset of `Variable` objects that are local to each
  machine. Usually used for temporarily variables, like counters.
  Note: use `tf.contrib.framework.local_variable` to add to this collection.

* `MODEL_VARIABLES`: the subset of `Variable` objects that are used in the
  model for inference (feed forward). Note: use
  `tf.contrib.framework.model_variable` to add to this collection.

* `TRAINABLE_VARIABLES`: the subset of `Variable` objects that will
  be trained by an optimizer. 更多细节详见`tf.compat.v1.global_variables`

* `SUMMARIES`: the summary `Tensor` objects that have been created in the
  graph. See
  `tf.compat.v1.summary.merge_all`
  for more details.

* `QUEUE_RUNNERS`: 用于为计算生成输入的`QueueRunner`对象，更多细节详见`tf.compat.v1.train.start_queue_runners`

* `MOVING_AVERAGE_VARIABLES`: the subset of `Variable` objects that will also
  keep moving averages.  更多细节详见`tf.compat.v1.moving_average_variables`

* `REGULARIZATION_LOSSES`: regularization losses collected during graph
  construction.

The following standard keys are _defined_, but their collections are **not**
automatically populated as many of the others are:

* `WEIGHTS`
* `BIASES`
* `ACTIVATIONS`

**File**: \tensorflow\lib\site-packages\tensorflow\python\framework\ops.py

**Type**:           type